# Numerical Initializers Development
Zakariya Abugrin - 04.09.2023

## Introductions

Numerical solutions consist of two steps as following:
1. Initialization: in this step a system of linear equations is created based on `Model` class using numerical initializers (`NumericalInitializer`).
2. Solving the system of linear equations using direct solvers, iterative solvers, or neurical solvers. 
3. Steps 1 and 2 are repeated until a final timestep is reached.

In this notebook, `NumericalInitializer` class is developed for `FDM`.

In [2]:
import reservoirflow as rf

print(rf.__version__)

0.1.0


In [5]:
grid = rf.grids.RegularCartesian(
    nx=4,
    ny=1,
    nz=1,
    dx=300,
    dy=350,
    dz=40,
    phi=0.27,
    kx=270,
    dtype="double",
)

fluid = rf.fluids.SinglePhase(mu=0.5, B=1, dtype="double")

model = rf.models.BlackOil(grid, fluid, dtype="double", pi=4000, verbose=True)
model.set_well(cell_id=4, q=-600, s=1.5, r=3.5)
model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
# model.compile(stype="numerical", method="fdm")
# model.compile(stype="analytical", method="-")
# model.compile(stype="neurical", method="pinn")

[info] the model was initialized.
[info] model compressibility (comp) was calculated.
[info] a well in cell 4 was set.
[info] boundary in cell 0 was set to constant pressure.
[info] boundary in cell 5 was set to constant rate.


In [6]:
model.get_cells_eq()

[info] cell id: 1
[info]    - Neighbors: [2]
[info]    - Boundaries: [0]
[info] Neighbor terms: -28.4004*p1 + 28.4004*p2
[info] Boundary terms: 227203.2 - 56.8008*p1
[info] Accumulation term: 0.0
[info] terms: {'f_terms': [-28.4004*p1 + 28.4004*p2, 227203.2 - 56.8008*p1], 'a_term': 0.0}
[info] Flow equation 1: Eq(85.2012*p1 - 28.4004*p2, 227203.2)
[info] cell id: 1
[info]      - lhs: defaultdict(<class 'int'>, {p1: 85.2012000000000, p2: -28.4004000000000})
[info]      - rhs: 227203.200000000
[info] cell id: 2
[info]    - Neighbors: [1 3]
[info]    - Boundaries: []
[info] Neighbor terms: 28.4004*p1 - 28.4004*p2
[info] Neighbor terms: -28.4004*p2 + 28.4004*p3
[info] Accumulation term: 0.0
[info] terms: {'f_terms': [28.4004*p1 - 28.4004*p2, -28.4004*p2 + 28.4004*p3], 'a_term': 0.0}
[info] Flow equation 2: Eq(28.4004*p1 - 56.8008*p2 + 28.4004*p3, 0.0)
[info] cell id: 2
[info]      - lhs: defaultdict(<class 'int'>, {p1: 28.4004000000000, p3: 28.4004000000000, p2: -56.8008000000000})
[info] 

{1: (defaultdict(int, {p1: 85.2012000000000, p2: -28.4004000000000}),
  227203.200000000),
 2: (defaultdict(int,
              {p1: 28.4004000000000,
               p3: 28.4004000000000,
               p2: -56.8008000000000}),
  0.0),
 3: (defaultdict(int,
              {p2: 28.4004000000000,
               p4: 28.4004000000000,
               p3: -56.8008000000000}),
  0.0),
 4: (defaultdict(int, {p3: 28.4004000000000, p4: -28.4004000000000}),
  600.000000000000)}

In [7]:
A = model.init_A(False)
d = model.init_d(False)
print(f"A:{A}")
print(f"d:{d}")

A:[[-28.4004  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]]
d:[[0.]
 [0.]
 [0.]
 [0.]]


In [8]:
model.get_matrices_vectorized(False, True)

(array([[-85.2012,  28.4004,   0.    ,   0.    ],
        [ 28.4004, -56.8008,  28.4004,   0.    ],
        [  0.    ,  28.4004, -56.8008,  28.4004],
        [  0.    ,   0.    ,  28.4004, -28.4004]]),
 array([[-227203.2],
        [      0. ],
        [      0. ],
        [    600. ]]))

In [9]:
model.verbose = False
model.init_matrices(True, True)

(<4x4 sparse matrix of type '<class 'numpy.float64'>'
 	with 10 stored elements in List of Lists format>,
 <4x1 sparse matrix of type '<class 'numpy.float64'>'
 	with 2 stored elements in List of Lists format>)

In [14]:
model.run(10, print_arrays=False)

[info] Simulation run started: 10 timesteps.


[step] 10: 100%|██████████| 10/10 [00:00<00:00, 344.77steps/s]

[info] Simulation run of 10 steps finished in 0.03 seconds.
[info] Material Balance Error: 1.693933882052079e-11.


In [15]:
model.get_df()

,Time,Q0,Q4,P0,P1,P2,P3,P4,Qw4,Pwf4
Step,,,,,,,,,,
0,0,0.0,0.0,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,0.0,4000.000000
1,1,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
2,2,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
3,3,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
4,4,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
5,5,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
6,6,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
7,7,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
8,8,600.0,-600.0,4000.0,3989.436768,3968.310305,3947.183842,3926.057379,-600.0,3898.992647
